In [1]:
import folium
import json
from shapely.geometry import shape, mapping, Point
from shapely.ops import unary_union

# Cargar el GeoJSON de pistas
with open("./puntosespera/runways.geojson", 'r', encoding='utf-8') as f:
    data = json.load(f)

# Diccionario para almacenar las geometrías combinadas por nombre de pista
runway_geometries = {}
properties_store = {}

for feature in data["features"]:
    runway_name = feature["properties"].get("RWY")
    geometry = shape(feature["geometry"])  # Convertir a objeto shapely
    
    if runway_name:
        if runway_name in runway_geometries:
            runway_geometries[runway_name].append(geometry)
        else:
            runway_geometries[runway_name] = [geometry]
            properties_store[runway_name] = feature["properties"]  # Guardar propiedades

# Combinar geometrías por pista y convertirlas en rectángulos
combined_features = []
runway_centroids = {}  # Almacenar centroides para búsqueda rápida

for runway_name, geometries in runway_geometries.items():
    combined_geometry = unary_union(geometries)  # Unir todas las geometrías
    min_rectangle = combined_geometry.minimum_rotated_rectangle  # Obtener rectángulo mínimo
    runway_centroids[runway_name] = min_rectangle.centroid  # Calcular centroide
    
    combined_features.append({
        "type": "Feature",
        "geometry": mapping(min_rectangle),  # Guardar el rectángulo en el GeoJSON
        "properties": properties_store[runway_name]
    })

# Crear un nuevo GeoJSON con las pistas en forma de rectángulos
filtered_data = {"type": "FeatureCollection", "features": combined_features}

# Guardar el nuevo GeoJSON
with open("./puntosespera/runways_rectangles.geojson", 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

# Cargar el GeoJSON de puntos de espera
with open("./puntosespera/holding_points.geojson", 'r', encoding='utf-8') as f:
    holding_data = json.load(f)

# Asignar cada punto de espera a la pista más cercana
for feature in holding_data["features"]:
    point = Point(feature["geometry"]["coordinates"])  # Convertir a objeto shapely
    closest_runway = min(runway_centroids, key=lambda rwy: point.distance(runway_centroids[rwy]))
    feature["properties"]["CLOSEST_RUNWAY"] = closest_runway  # Asignar pista más cercana

# Guardar el GeoJSON actualizado con la información de la pista más cercana
with open("./puntosespera/holding_points_updated.geojson", 'w', encoding='utf-8') as f:
    json.dump(holding_data, f, ensure_ascii=False, indent=4)

# Crear el mapa centrado en Madrid
madrid_map = folium.Map(location=[40.5, -3.55], zoom_start=12, tiles="cartodbpositron")

# Agregar las pistas simplificadas al mapa
folium.GeoJson(
    filtered_data,
    name="Runways (Rectangles)",
    tooltip=folium.GeoJsonTooltip(fields=["RWY", "NAME_TXT_ADHP"], aliases=["Runway:", "Airport:"])
).add_to(madrid_map)

# Agregar los puntos de espera al mapa
for feature in holding_data["features"]:
    coords = feature["geometry"]["coordinates"]
    folium.Circle(
        location=[coords[1], coords[0]],  # Folium usa latitud, longitud
        radius=20,  # Radio en metros (para diámetro de 10m)
        color="black",
        fill=True,
        fill_color="black",
        fill_opacity=0.5,
        popup=folium.Popup(f"Designator: {feature['properties'].get('DESIGNATOR', 'N/A')}\n"
                           f"Closest Runway: {feature['properties'].get('CLOSEST_RUNWAY', 'N/A')}\n"
                           f"Remarks: {feature['properties'].get('REMARKS_TXT', 'N/A')}", max_width=300)
    ).add_to(madrid_map)

# Mostrar el mapa
madrid_map


FileNotFoundError: [Errno 2] No such file or directory: './puntosespera/runways.geojson'

In [66]:
import pandas as pd
from shapely.geometry import shape

# Crear DataFrame de pistas
runway_data = []
for feature in filtered_data["features"]:
    properties = feature["properties"]
    geometry = shape(feature["geometry"])  # Convertir a objeto shapely
    runway_data.append({
        "Runway": properties.get("RWY", "N/A"),
        "Airport": properties.get("NAME_TXT_ADHP", "N/A"),
        "Geometry": geometry
    })

df_runways = pd.DataFrame(runway_data)

# Crear DataFrame de puntos de espera
holding_data_list = []
for feature in holding_data["features"]:
    properties = feature["properties"]
    coords = feature["geometry"]["coordinates"]
    holding_data_list.append({
        "Designator": properties.get("DESIGNATOR", "N/A"),
        "Closest Runway": properties.get("CLOSEST_RUNWAY", "N/A"),
        "Remarks": properties.get("REMARKS_TXT", "N/A"),
        "Latitude": coords[1],
        "Longitude": coords[0]
    })

df_holding_points = pd.DataFrame(holding_data_list)

# Mostrar los primeros registros
df_runways.head(), df_holding_points.head()


(    Runway                              Airport  \
 0  18L/36R  MADRID/Adolfo Suárez Madrid-Barajas   
 1  14L/32R  MADRID/Adolfo Suárez Madrid-Barajas   
 2  18R/36L  MADRID/Adolfo Suárez Madrid-Barajas   
 3  14R/32L  MADRID/Adolfo Suárez Madrid-Barajas   
 
                                             Geometry  
 0  POLYGON ((-3.560314698993372 40.53621495356422...  
 1  POLYGON ((-3.5606164431646263 40.4964888292004...  
 2  POLYGON ((-3.5753122209006754 40.5330356794210...  
 3  POLYGON ((-3.545264836113798 40.45570158442552...  ,
   Designator Closest Runway                                Remarks   Latitude  \
 0         Z1        18R/36L  Con iluminación LED. // LED ligthing.  40.490653   
 1        KA6        14L/32R  Con iluminación LED. // LED ligthing.  40.472076   
 2        KA8        14L/32R  Con iluminación LED. // LED ligthing.  40.466623   
 3         K3        14L/32R  Con iluminación LED. // LED ligthing.  40.494123   
 4         K2        14L/32R  Con iluminación L

In [70]:
df_runways["Runway"] = df_runways["Runway"].map(lambda x: f"{x[-3:]}/{x[:3]}")

In [82]:
df_runways = df_runways[["Runway", "Geometry"]]

In [78]:
!pip install geopandas

  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 9.6 MB/s eta 0:00:00 0:00:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 46.5 MB/s eta 0:00:00


In [84]:
import geopandas as gpd

# Guardar pistas en GeoJSON
gdf_runways = gpd.GeoDataFrame(df_runways, geometry="Geometry")
gdf_runways.to_file("runways_processed.geojson", driver="GeoJSON")

/opt/anaconda3/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [92]:
df_holding_points["Runway"] = df_holding_points["Closest Runway"].map(lambda x: f"{x[-3:]}/{x[:3]}")

In [96]:
df_runways

,Runway,Geometry
0,36R/18L,POLYGON ((-3.560314698993372 40.53621495356422...
1,32R/14L,POLYGON ((-3.5606164431646263 40.4964888292004...
2,36L/18R,POLYGON ((-3.5753122209006754 40.5330356794210...
3,32L/14R,POLYGON ((-3.545264836113798 40.45570158442552...


In [102]:
df_holding_points = df_holding_points[["Designator", "Runway", "Latitude", "Longitude"]]

In [104]:
df_holding_points

,Designator,Runway,Latitude,Longitude
0,Z1,36L/18R,40.490653,-3.573093
1,KA6,32R/14L,40.472076,-3.537524
2,KA8,32R/14L,40.466623,-3.536653
3,K3,32R/14L,40.494123,-3.558960
4,K2,32R/14L,40.494596,-3.559326
5,K1,32R/14L,40.495555,-3.560411
6,Y1,36R/18L,40.499431,-3.560656
7,Y2,36R/18L,40.500298,-3.560646
8,Y3,36R/18L,40.501184,-3.560660
9,Y7,36R/18L,40.533392,-3.560800


In [108]:
import geopandas as gpd

# Crear GeoDataFrame con geometría
gdf_holding_points = gpd.GeoDataFrame(
    df_holding_points, geometry=gpd.points_from_xy(df_holding_points.Longitude, df_holding_points.Latitude)
)

# Eliminar columnas Longitude y Latitude
gdf_holding_points = gdf_holding_points.drop(columns=["Longitude", "Latitude"])

# Guardar en GeoJSON
gdf_holding_points.to_file("holding_points_processed.geojson", driver="GeoJSON")

/opt/anaconda3/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
